In [ ]:
import sys

import torch.nn as nn
import torch.nn.functional as F
import torch as torch
import torch.utils.data as TData
from torch import optim
from torch.nn.parameter import Parameter
from torchsummary import summary
from torch.autograd import Variable
import torchvision.transforms as transforms
import math
from scipy.stats import chi2
from scipy.stats import t as gosset_t

import matplotlib.pyplot as plt

from PIL import Image
import plotly.express as px
import time
import numpy as np
import os
import pandas as pd
from pytorch_model_summary import summary
import random


from tqdm import tqdm
from einops import rearrange
import copy

In [ ]:
import HeidelbergTraining.UNet.UNet_functions as unet_functions
import HeidelbergTraining.UNet.UNet_model as unet_models

In [ ]:
N_CLASSES = 13
img_size = (1, 512, 512)  # (channels, width, height)
LAYER_NAMES_13 = [
    "ILM", 
    "RNFL",
    "GCL", 
    "IPL", 
    "INL", 
    "OPL", 
    "ELM", 
    "PR1", 
    "PR2", 
    "RPE", 
    "Interlayer holes", 
    "BG top", 
    "BG bottom"
]
LAYER_NAMES_14 = [
    "ILM", 
    "RNFL",
    "GCL", 
    "IPL", 
    "INL", 
    "OPL", 
    "ELM", 
    "PR1", 
    "PR2", 
    "RPE", 
    "Collapsed Layers",
    "Cycsts",
    "Vitreous", 
    "Choroid/Sclera"
]
LAYER_NAMES_15 = [
    "ILM",   
    "RNFL", 
    "GCL", 
    "IPL", 
    "INL", 
    "OPL", 
    "ONL", 
    "ELM", 
    "PR1", 
    "PR2", 
    "RPE", 
    "BM", 
    "Interlayer holes", 
    "BG top", 
    "BG bottom"
]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights_13 = Variable(torch.tensor([9, 6, 3, 3, 3, 3, 3, 3, 6, 6, 6, 1, 1]).float()).to(device)
class_weights_15 = Variable(torch.tensor([9, 3, 3, 3, 3, 3, 3, 3, 6, 6, 6, 1, 1, 9, 9]).float()).to(device)
total_path = "./"

total_mean = 87.1074833179442
total_std = 63.13706830280864


In [ ]:
print(len("images_input_unlabeled0000.png: "))

In [ ]:
with open("./HeidelbergTraining/images_input_unlabeled_paths.txt", "r") as unlabeled_original_locations:
    data = unlabeled_original_locations.readlines()
    data = [elem[32:].strip() for elem in data]
    print(len(set(data)))

In [ ]:
def make_list(path):
    ret = []
    for root,dirs,files in os.walk(path):
        ret = sorted([path + "/" + f for f in files if f[-4:] == ".png"])
        if len(ret) > 0:
            break
    return ret

image_input_labeled_path = os.path.join(total_path, "HeidelbergTraining/images_input_labeled")
image_mask_path = os.path.join(total_path, "HeidelbergTraining/images_masks")
baseline_img_path = os.path.join(total_path, "HeidelbergTraining/images_baseline/baseline_images")
baseline_mask_path = os.path.join(total_path, "HeidelbergTraining/images_baseline/baseline_masks")
baseline_full_img_path = os.path.join(total_path, "HeidelbergTraining/images_baseline/baseline_full_image")


# go to images_input_labeled_paths.txt, images_masks_paths.txt for more info on these numbers
# 80:10:10 split at patient level
num_diseased = 292
diseased_train_cutoff = 230
normal_train_cutoff = 1381
diseased_valid_cutoff = 262
normal_valid_cutoff = 1521

images_all_labeled = make_list(image_input_labeled_path)
masks_all = make_list(image_mask_path)

image_list_train = images_all_labeled[num_diseased:normal_train_cutoff]
mask_list_train_13 = masks_all[num_diseased:normal_train_cutoff]

dimgs = images_all_labeled[:diseased_train_cutoff]
dmasks = masks_all[:diseased_train_cutoff]

while len(image_list_train) < (2 * (normal_train_cutoff - num_diseased)):
    image_list_train.extend(dimgs)
    mask_list_train_13.extend(dmasks)

del dimgs
del dmasks

d_image_list_cv = images_all_labeled[diseased_train_cutoff:diseased_valid_cutoff]
d_mask_list_cv_13 = masks_all[diseased_train_cutoff:diseased_valid_cutoff]

d_image_list_test = []
d_mask_list_test_13 = []

with open("./HeidelbergTraining/images_baseline_paths.txt", "r") as baseline_info:
    data = baseline_info.readlines()
    d_mask_list_test_13 = [elem[elem.find(":")+2:].strip() for elem in data]
    d_image_list_test = ["./HeidelbergTraining/images_input_labeled/images_input_labeled%s.png"%elem[elem.find(".png")-4:elem.find(".png")] for elem in d_mask_list_test_13]

image_list_train = sorted(image_list_train)
mask_list_train_13 = sorted(mask_list_train_13)

d_image_list_cv = sorted(d_image_list_cv)
d_mask_list_cv_13 = sorted(d_mask_list_cv_13)

d_image_list_test = sorted(d_image_list_test)
d_mask_list_test_13 = sorted(d_mask_list_test_13)

baseline_images = make_list(baseline_img_path)
baseline_masks = make_list(baseline_mask_path)
baseline_full_images = make_list(baseline_full_img_path)


In [ ]:
print(len(images_all_labeled))

In [ ]:
# set_mask_train = set(mask_list_train_13)
# percentages = np.full((13,), 0).astype("float32")
# for i, path in enumerate(set_mask_train):
#     mask = np.array(Image.open(path))
#     for label in range(13):
#         percentages[label] += (np.sum(mask.ravel() == label) / (mask.shape[0] * mask.shape[1]))
    
# percentages /= len(set_mask_train)
# # percentages = percentages
# print(percentages)

In [ ]:
# for i, (img_path, mask_path) in enumerate(zip(baseline_images, baseline_masks)):
#     if i > 2:
#         break
#     fig,arr = plt.subplots(1,2,figsize=(14,10))
#     img = np.array(Image.open(img_path).convert("L"))
#     mask = np.array(Image.open(mask_path).convert("L"))
    
#     arr[0].imshow(img)
#     arr[1].imshow(mask)
#     fig.show()
#     plt.show()

In [ ]:
# print((np.max(percentages) / percentages).astype(int))

In [ ]:
# total_mean = 0
# total_squared_mean = 0

# set_iter_train = set(image_list_train)

# for i, path in enumerate(set_iter_train):
#     img = np.array(Image.open(path)).astype("int32")
    
    
#     total_mean += np.mean(img.ravel())
#     total_squared_mean += np.mean(np.square(img.ravel()))

# total_squared_mean /= len(set_iter_train)
# total_mean /= len(set_iter_train)

# total_std = np.sqrt((total_squared_mean - np.square(total_mean)))

# print(total_mean)
# print(total_std)


In [ ]:
# total_mean = 87.1074833179442
# total_std = 63.13706830280864
# set_iter_train = set(image_list_train)

# mean_check = 0
# squared_mean_check = 0

# for i, path in enumerate(set_iter_train):
#     img = (np.array(Image.open(path)) - total_mean) / total_std
#     mean_check += np.mean(img.ravel())
#     squared_mean_check += np.mean(np.square(img.ravel()))
    

# mean_check /= len(set_iter_train)
# squared_mean_check /= len(set_iter_train)

# std_check = np.sqrt(squared_mean_check - np.square(mean_check))
# print(mean_check)
# print(squared_mean_check)
    

In [ ]:
print("lengths:")
print("image list train:\t %s"%len(image_list_train))
print("d image list test:\t %s"%len(d_image_list_test))
print("d image list cv:\t %s"%len(d_image_list_cv))
# print("unlabeled image list:\t %s"%len(uimage_list_full))

In [ ]:
def show_output(
    eval_model, 
    name="best_model",
    image_path=None, 
    mask_path=None, 
    verbose=True,
    sig_figs=6,
    ignore_0=False,
    Wnet=False,
    prev_model=False
):
    count = 2
    image = None
    mask = None
    output = None
    
    eval_model.eval()
    
    n_class = list(range(N_CLASSES))
    if ignore_0:
        n_class = n_class[:10] + n_class[11:]
    
    display_image = np.array(Image.open(image_path).convert("L"))
    
    image = np.array(Image.open(image_path).convert("L"))
    total_mean = 87.1074833179442
    total_std = 63.13706830280864
    image = ((image) - total_mean) / total_std
    image = image[np.newaxis, np.newaxis, :]
    image = torch.from_numpy(image)
    image = Variable(image.float()).to(device)
    
#     print(name + "\n---------------------")
#     print(image.shape)
    
    if Wnet:
        output = prev_model(image)
        output = torch.argmax(rearrange(output[0].detach(), 'c h w -> h w c'), dim=-1).cpu().numpy()
        output = output[np.newaxis, np.newaxis, :]
        
        image = display_image[np.newaxis, np.newaxis, :]
        image = np.concatenate((image, output), axis=1)
        image = torch.from_numpy(image)
        image = Variable(image.float()).to(device)
        
        output = eval_model(image)
        
    else:
        output = eval_model(image)
    
    print(output.shape)
    output = torch.argmax(rearrange(output[0].detach(), 'c h w -> h w c'), dim=-1).cpu().numpy()
        
    
    if mask_path is not None:
        count += 1
        mask = np.array(Image.open(mask_path).convert("L"))
    
    fig, arr = plt.subplots(1, count, figsize=(14, 10))
    arr[0].imshow(display_image)
    arr[0].set_title("(%s) - input image"%name)
    arr[1].imshow(output)
    arr[1].set_title("(%s) - output image"%name)
    if count > 2:
        arr[2].imshow(mask*20)
        arr[2].set_title("(%s) - mask"%name)
    
    if verbose:
        print("image path: %s"%image_path)
        print("image shape: %s, %s"%display_image.shape)
        print("output shape: %s, %s"%output.shape)
        if count > 2:
            print("mask shape: %s, %s"%mask.shape)
            iou_c, mean_c = unet_functions.get_iou(mask, output, n_classes=n_class)
            print("IOU: ")
            
            names = LAYER_NAMES_13
            if ignore_0:
                names = LAYER_NAMES_13[:10]+LAYER_NAMES_13[11:]
            
            for i,l in enumerate(names):
                pad = "\t"
                if len(l) < 6:
                    pad += "\t"
                if len(l) < 12:
                    pad += "\t"
                    
                num = str(iou_c[i])
                num = num[:sig_figs]
                    
                print(l + ": " + pad + "%s"%num)
            print("\nMean IOU")
            m = str(mean_c)[:sig_figs]
            print("%s"%m)
    print("---------------------")

In [ ]:
class CustomDataset(TData.Dataset):
    def __init__(
        self, 
        image_paths, 
        diseased_paths, 
        augs=None, 
        image_specs=None, 
        n_classes=13,
        prev_model=None
    ):
        super(CustomDataset, self).__init__()
        
        sample = []
        
        for i,d in zip(image_paths, diseased_paths):
            sample.append((i,d))
        
        self.samples = sample
        self.channels = image_specs[0]
        self.rows = image_specs[1]
        self.col = image_specs[2]
        self.classes = n_classes
        self.augments = augs
        self.model = prev_model
        self.total_std = 63.13706830280864  # calculated on training set
        self.total_mean = 87.1074833179442 # calculated on training set
    
    def __getitem__(self, index):
        image = Image.open(self.samples[index][0]).convert("L")
        mask = np.zeros((self.rows, self.col), dtype=np.float32)
        
        ssl = True if self.samples[index][1] is None else False
        
        if not ssl:
            mask = Image.open(self.samples[index][1]).convert("L")
            
        if self.augments is not None:
            seed = random.random() * 100
            random.seed(seed)
            image = self.augments(image)
            if not ssl:
                random.seed(seed)
                mask = self.augments(mask)
        
        image = np.array(image)
        mean = np.mean(image)
        std = np.std(image)
        image = ((image - mean) / std)[np.newaxis, :]
        
        if self.model is not None:
            self.model.train()
            image2 = image[np.newaxis, :]
            image2 = torch.from_numpy(image2)
            image2 = Variable(image2.float()).to(device)
            
            image2 = self.model(image2)
            image2 = torch.argmax(rearrange(image2[0].detach(), 'c h w -> h w c'), dim=-1).cpu().numpy()
            image2 = image2[np.newaxis, :]
            
            image = np.concatenate((image, image2), axis=0)
        
        if not ssl:
            mask = np.array(mask, dtype=np.float32)

        return image, mask
    
    def __len__(self):
        return len(self.samples)

In [ ]:
standard_unet_model = unet_models.unet_model(
    32, 
    num_downs=4, 
    n_classes=13, 
    input_channels=1, 
    name="standard unet",
).to(device)
state_dict = torch.load("./HeidelbergTraining/saved_model_pytorch/standard unet_iter1")
standard_unet_model.load_state_dict(state_dict)

aug1 = transforms.RandomChoice([
    transforms.RandomHorizontalFlip(),
    transforms.Compose([
        transforms.RandomCrop(size=(256, 256)),
        transforms.Resize((512, 512), interpolation=Image.NEAREST),
        transforms.RandomHorizontalFlip()
    ]),
    transforms.Compose([
        transforms.RandomRotation(15),
        transforms.CenterCrop((256, 256)),
        transforms.Resize((512, 512), interpolation=Image.NEAREST),
        transforms.RandomHorizontalFlip()
    ]),
    transforms.Compose([
        transforms.RandomCrop((256, 256)),
        transforms.Resize((512, 512), interpolation=Image.NEAREST),
        transforms.RandomHorizontalFlip()
    ])
])

iter_train_standard = TData.DataLoader(
    CustomDataset(
        image_list_train, 
        mask_list_train_13, 
        image_specs=img_size, 
        prev_model=None,
        augs=None
    )
)

iter_d_valid_standard = TData.DataLoader(
    CustomDataset(d_image_list_cv, d_mask_list_cv_13, image_specs=img_size, prev_model=None)
)

iter_d_test_standard = TData.DataLoader(
    CustomDataset(d_image_list_test, d_mask_list_test_13, image_specs=img_size, prev_model=None)
)


In [ ]:
double_unet_model = unet_models.unet_model(
    32, 
    num_downs=4, 
    n_classes=13, 
    input_channels=2, 
    name="prev_net",
).to(device)

state_dict = torch.load("./HeidelbergTraining/saved_model_pytorch/double unet_iter1")
double_unet_model.load_state_dict(state_dict)

iter_train_double = TData.DataLoader(
    CustomDataset(image_list_train, mask_list_train_13, image_specs=img_size, prev_model=standard_unet_model)
)

iter_d_valid_double = TData.DataLoader(
    CustomDataset(d_image_list_cv, d_mask_list_cv_13, image_specs=img_size, prev_model=standard_unet_model)
)

iter_d_test_double = TData.DataLoader(
    CustomDataset(d_image_list_test, d_mask_list_test_13, image_specs=img_size, prev_model=standard_unet_model)
)

In [ ]:
# def add_13_label(output, image):
#     avg_pooler = nn.AvgPool2d(4, stride=2).to(device)
#     reduced_output = (avg_pooler(output[None, None, :, :].float()))[0][0].detach().cpu().numpy()
#     reduced_image = (avg_pooler(image[:,0:1,:,:]))[0][0].detach().cpu().numpy()
    
#     threshold = np.percentile(reduced_image, 25)

#     a = [reduced_image < threshold][0].astype(np.int8)
#     b = [reduced_output == 0][0].astype(np.int8)
#     b = ((a + b) // 2).astype(bool)

#     yvals, xvals = np.where(b)
#     for (x,y) in zip(xvals, yvals):
#         output[y*2:y*2+5,x*2:x*2+5] = 13
    
#     return output

# for i, img_path in enumerate(d_image_list_test):
# #     if i > 0:
# #         break
#     image = np.array(Image.open(img_path))
    
#     mean = np.mean(image.ravel())
#     std = np.std(image.ravel())
    
#     input_image_standard = torch.from_numpy(((image - mean) / std)[np.newaxis, :])[None, :, :, :]
#     output_standard = standard_unet_model(Variable(input_image_standard).float().to(device))[0].detach().cpu().numpy()
#     output_standard = np.argmax(output_standard.transpose(1,2,0), axis=-1).astype("uint8")
    
#     input_image_double = torch.cat((input_image_standard.float(), torch.from_numpy(output_standard)[None, None, :, :].float()), dim=1)
#     output_double = double_unet_model(Variable(input_image_double).to(device))[0].detach().cpu().numpy()
#     output_double = np.argmax(output_double.transpose(1,2,0), axis=-1).astype("uint8")
    
#     output_standard = add_13_label(torch.from_numpy(output_standard), input_image_standard).numpy()
#     output_double = add_13_label(torch.from_numpy(output_double), input_image_double).numpy()
    
#     blend_alpha = 0.4
    
#     image = Image.fromarray(image.astype("uint8"))
#     blended_img_standard = Image.blend(Image.fromarray(output_standard * 19), image, blend_alpha)
#     blended_img_double = Image.blend(Image.fromarray(output_double * 19), image, blend_alpha)
    
#     fig,arr = plt.subplots(1,5,figsize=(14,10))
#     for a in arr:
#         a.axes.get_xaxis().set_visible(False)
#         a.axes.get_yaxis().set_visible(False)
        
#     arr[0].imshow(np.array(image))
#     arr[1].imshow(output_standard)
#     arr[2].imshow(np.array(blended_img_standard))
#     arr[3].imshow(output_double)
#     arr[4].imshow(np.array(blended_img_double))
#     fig.show()
#     plt.show()

In [ ]:
def calc_iou(image, mask, n_class=0):
    image = image.astype("int32")
    mask = mask.astype("int32")
    actual_positives = mask == n_class
    predicted_positives = image == n_class
    
    intersection = np.sum(np.logical_and(actual_positives, predicted_positives))
    union = np.sum(np.logical_or(actual_positives, predicted_positives))
    
    if union == 0 and intersection == 0:
        return 1

    iou = intersection/union
    return iou

def get_iou(output, mask, n_classes=list(range(14))):
    ious = []
    for i in n_classes:
        if i < 10:
            ious.append(calc_iou(output, mask, i+1))
        elif i == 10:
            ious.append(calc_iou(output, mask, 0))
        elif i == 11 and len(n_classes) == 14:
            ious.append(calc_iou(output, mask, 13))
        elif len(n_classes) == 14:
            ious.append(calc_iou(output, mask, i-1))
        else:
            ious.append(calc_iou(output, mask, i))
        
    mean_iou = sum(ious) / len(n_classes)
    
    return ious, mean_iou

def evaluate_model2(eval_model, iter_set, show=False, name="DL_pytorch", ignore_0=False, n_classes=list(range(N_CLASSES))):
#     eval_model.train(False)
    ious = [[] for i in range(len(n_classes))]
    means = []
    
#     eval_model.eval()
    
    for i, (image, mask) in enumerate(iter_set):
        image,mask = Variable(image.float()).to(device), Variable(mask.long()).to(device)
        output = eval_model(image)
        
        output = torch.argmax(rearrange(output[0].detach(), 'c h w -> h w c'), dim=-1)
        
        avg_pooler = nn.AvgPool2d(4, stride=2).to(device)
    
        reduced_mask = (avg_pooler(mask[None, :, :, :].float()))[0][0].detach().cpu().numpy()
        reduced_output = (avg_pooler(output[None, None, :, :].float()))[0][0].detach().cpu().numpy()
        reduced_image = (avg_pooler(image[:,0:1,:,:]))[0][0].detach().cpu().numpy()
        
        
        output = output.cpu().numpy()
        image = image[0][0].cpu().numpy()
        mask = mask[0].cpu().numpy()

        threshold = np.percentile(reduced_image, 25)

        a = [reduced_image < threshold][0].astype(np.int8)
        b = [reduced_output == 0][0].astype(np.int8)
        b = ((a + b) // 2).astype(bool)

        a = [reduced_image < threshold][0].astype(np.int8)
        c = [reduced_mask == 0][0].astype(np.int8)
        c = ((a + c) // 2).astype(bool)
        
        yvals, xvals = np.where(c)
        for (x,y) in zip(xvals, yvals):
            mask[y*2:y*2+5,x*2:x*2+5] = 13

        yvals, xvals = np.where(b)
        for (x,y) in zip(xvals, yvals):
            output[y*2:y*2+5,x*2:x*2+5] = 13
        
        if show:
            fig, arr = plt.subplots(1, 3, figsize=(14,10))
            arr[0].imshow(mask)
            arr[0].set_title("gt %s"%i)
            arr[1].imshow(output)
            arr[1].set_title("prediction %s"%i)
            arr[2].imshow(image)
            arr[2].set_title("input %s"%i)
            fig.show()
            plt.show()
        
        iou_c, mean_c = get_iou(mask, output, n_classes=range(14))
        
        means.append(mean_c)
        for j in range(len(ious)):
            if j >= len(iou_c):
                ious[j].append(0)
            else:
                ious[j].append(iou_c[j])
                
    means = sum(means)/len(means)
    iou_data = []
    for layer_name,layer in enumerate(ious):
        iou_data.extend([(name, layer_name, layer_i) for layer_i in layer])
        
    iou_data = pd.DataFrame(iou_data, columns=["Class", "layer", "IOU"])
    
    return iou_data,means


In [ ]:
def get_baseline_ious(baseline_img_path_list, baseline_mask_path_list):
    ious = [[] for i in range(14)]
    means = []
    
    for i, (img_path, mask_path) in enumerate(zip(baseline_img_path_list, baseline_mask_path_list)):
        baseline_img = np.array(Image.open(img_path).convert("L"))
        baseline_mask = np.array(Image.open(mask_path).convert("L"))
        
        iou_c, mean_c = get_iou(baseline_mask, baseline_img, n_classes=range(14))
        means.append(mean_c)
        for j in range(len(ious)):
            if j == 10 or j == 11:
                continue
            ious[j].append(iou_c[j])
#             if j >= len(iou_c):
#                 ious[j].append(0)
#             else:
#                 ious[j].append(iou_c[j])
        ious[10].append(0)
        ious[11].append(0)
            
    means = sum(means)/len(means)
    
    iou_data = []
    name = "Heidelberg Auto"
    for layer_name,layer in enumerate(ious):
        iou_data.extend([(name, layer_name, layer_i) for layer_i in layer])
        
    iou_data = pd.DataFrame(iou_data, columns=["Class", "layer", "IOU"])
    
    return iou_data,means
        

In [ ]:
dl_name = "Semi-Supervised"
standard_name = "Standard UNet"
double_name = "Double UNet"
baseline_name = "Heidelberg Auto"

In [ ]:
iou_baseline, mean_baseline = get_baseline_ious(baseline_images, baseline_masks)


In [ ]:
standard_unet_model = standard_unet_model.train()
iou_standard, mean_standard = evaluate_model2(standard_unet_model, iter_d_test_standard, name=standard_name, show=False, n_classes=list(range(14)))


In [ ]:
double_unet_model.train()
iou_double, mean_double = evaluate_model2(double_unet_model, iter_d_test_double, name=double_name, show=False, n_classes=list(range(14)))


In [ ]:
iou_cps = pd.read_csv("./HeidelbergTraining/CPS_data.csv")
mean_cps = np.mean(iou_cps["IOU"])
iou_cps["Class"] = dl_name

In [ ]:
ious = iou_cps.append(iou_standard.append(iou_double.append(iou_baseline)))

# ious = iou_standard.append(iou_double)

In [ ]:
def plot_dict(iou):
    fig = px.box(iou, x="layer", y="IOU", color="Class")
    fig.update_layout(
        xaxis = dict(
            tickmode = 'array',
            tickvals = list(range(14)),
            ticktext = LAYER_NAMES_14
        ),
        boxgap=.7,
    #         title="IOUs of different model architectures on the diseased test set by layer",
        title="IOU comparison of different models on the diseased test set",
        legend=dict(
            y=-0.35,
            x=0.01
        ),
        height=650
    )
    fig.show()

plot_dict(ious)
print("Standard mean IOU:\t\t %10f"%mean_standard)
print("Double mean IOU:\t\t %10f"%mean_double)
print("Heidelberg Auto mean IOU:\t %10f"%mean_baseline)
print("CPS mean IOU:\t\t\t %10f"%mean_cps)

In [ ]:
mean_ious = []
order = [dl_name, standard_name, double_name, baseline_name]
for layer in np.unique(ious["layer"]):
    layer_ious = ious[ious["layer"] == layer]
    
    l_iou = [LAYER_NAMES_14[layer]]
    for method in order:
        l_iou.append(np.mean(layer_ious[layer_ious["Class"] == method]["IOU"]))
    
    mean_ious.append(tuple(l_iou))
mean_ious = pd.DataFrame(mean_ious, columns=(["layer"] + order))
# print(mean_ious)

In [ ]:
mean_ious.to_csv("./HeidelbergTraining/IOU_tables.csv")

In [ ]:
df3 = ious
df3_ious = np.array(df3["IOU"])
df3_dice = (2 * df3_ious) / (1 + df3_ious)
df3 = df3.replace(to_replace=df3_ious, value=df3_dice)
df3 = df3.rename(columns={"IOU":"Dice"})

In [ ]:
fig = px.box(df3, x="layer", y="Dice", color="Class")
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(14)),
        ticktext = LAYER_NAMES_14
    ),
    boxgap=.7,
#         title="IOUs of different model architectures on the diseased test set by layer",
    title="Dice comparison of different models on the diseased test set",
    legend=dict(
        y=-0.35,
        x=0.01
    ),
    height=650
)
fig.show()

In [ ]:
mean_dices = []
order = [dl_name, standard_name, double_name, baseline_name]
for layer in np.unique(df3["layer"]).astype("uint8"):
    layer_dices = df3[df3["layer"] == layer]
    
    l_dice = [LAYER_NAMES_14[layer]]
    for method in order:
        l_dice.append(np.mean(layer_dices[layer_dices["Class"] == method]["Dice"]))
    
    mean_dices.append(tuple(l_dice))
mean_dices = pd.DataFrame(mean_dices, columns=(["layer"] + order))
# print(mean_dices)

In [ ]:
mean_dices.to_csv("./HeidelbergTraining/Dice_table.csv")

In [ ]:
def df_to_iou_list(iou_df, layer=0):
    rows = iou_df[iou_df["layer"] == layer]
    return np.array(rows["IOU"])

In [ ]:
def interval(iou_list, alpha=0.05):
    num = len(iou_list)
    sd = np.std(iou_list)
    mean = np.mean(iou_list)
    qtiles = np.array([-1, 1]) * gosset_t.ppf(q=[1 - alpha / 2], df = num-1)
    return ((qtiles * sd) / math.sqrt(num))

In [ ]:
def plot_bootstrap(bootstrap_data, num_indices=100):
    bootstrap_data.sort()
    x_start = int(bootstrap_data[0] * 100) - 1
    x_end = int(bootstrap_data[-1] * 100) + 1

    step = (x_end - x_start) / num_indices
    x = np.arange(x_start, x_end+1, step) / 100

    step = x[1] - x[0]
    y = []
    for i in range(len(x)):
        y.append(np.sum(np.logical_and(bootstrap_data > x[i] - step / 2, bootstrap_data < x[i] + step / 2)))
    y = np.array(y) / len(bootstrap_data)

    mean = np.sum(np.multiply(x,y))
    std = math.sqrt(np.sum(np.multiply(np.square(x), y)) - (mean * mean))
    
    plt.plot(x, y)

In [ ]:
def calc_bootstrap(data_list, num_reps=10000, verbose=False):
    bootstrap_distribution = []
    mean = []
    std = []
    bootstrap_distribution = []
    
    bootstrap_distribution.append(np.mean(data_list))
    
    for j in range(num_reps):
        sample_distro = []
        for k in range(len(data_list)):
            sample_distro.append(data_list[random.randint(0, len(data_list) - 1)])
        bootstrap_distribution.append(np.mean(sample_distro))
        
    if verbose:
        plot_bootstrap(bootstrap_distribution, num_indices=100)
    
    return bootstrap_distribution

In [ ]:
def df_to_dice_list(iou_df, layer=0):
    iou_list = df_to_iou_list(iou_df, layer=layer)
    return np.divide((iou_list * 2), (iou_list + 1))

In [ ]:
order = [dl_name, standard_name, double_name, baseline_name]

for method in order:
    iou_method = ious[ious["Class"] == method]
    iou_method_list = np.array(iou_method["IOU"])
    distro = calc_bootstrap(iou_method_list)
    print(1.96*np.std(distro))
    print(method)
    print()

In [ ]:
order = [dl_name, standard_name, double_name, baseline_name]

for method in order:
    dice_method = df3[df3["Class"] == method]
    dice_method_list = np.array(dice_method["Dice"])
    dice_method_list = np.divide((dice_method_list * 2), (dice_method_list + 1))
    distro = calc_bootstrap(iou_method_list)
    print(1.96*np.std(distro))
    print(method)
    print()

In [ ]:
order = [dl_name, standard_name, double_name, baseline_name]

confidence_intervals = []
for i in range(14):
    c_i = []
    for method in order:
        iou_method = ious[ious["Class"] == method]
        iou_method_list = df_to_iou_list(iou_method, layer=i)
        distro = calc_bootstrap(iou_method_list)
        c_i.append(1.96*np.std(distro))
    
    confidence_intervals.append(tuple(c_i))

confidence_intervals = pd.DataFrame(confidence_intervals, columns=order)

In [ ]:
order = [dl_name, standard_name, double_name, baseline_name]

confidence_intervals_dice = []
for i in range(14):
    print(i)
    c_i = []
    for method in order:
        dice_method = df3[df3["Class"] == method]
        dice_method_list = np.array(dice_method[dice_method["layer"] == i]["Dice"])
        distro = calc_bootstrap(dice_method_list)
        c_i.append(1.96*np.std(distro))
    
    confidence_intervals_dice.append(tuple(c_i))

confidence_intervals_dice = pd.DataFrame(confidence_intervals_dice, columns=order)

In [ ]:
order = [dl_name, standard_name, double_name, baseline_name]

confidence_intervals_dice2 = []
for i in range(14):
    print(i)
    c_i = []
    for method in order:
        dice_method = ious[ious["Class"] == method]
        dice_method_list = df_to_dice_list(dice_method, layer=i)
        distro = calc_bootstrap(dice_method_list)
        c_i.append(1.96*np.std(distro))
    
    confidence_intervals_dice2.append(tuple(c_i))

confidence_intervals_dice2 = pd.DataFrame(confidence_intervals_dice2, columns=order)

In [ ]:
for method in order:
    print(method)
    for elem in np.array(confidence_intervals_dice[method]):
        print(elem)
    
    print()